

# Regularization and the bias-variance trade-off

**Over-fitting** is a constant danger with machine learning models. Over-fit models fit the training data well. However, an over-fit model will not **generalize**. A model that generalizes is a model which exhibits good performance on data cases beyond the ones used in training. Models that generalize will be useful in production. 

As a general rule, an over-fit model has learned the training data too well. The over-fitting likely involved learning noise present in the training data. The noise in the data is random and uninformative. When a new data case is presented to such a model it may produce unexpected results since the random noise will be different. 

So, what is one to do to prevent over-fitting of machine learning models? The most widely used set of tools for preventing over-fitting are known as **regularization methods**. Regularization methods take a number of forms, but all have the same goal, to prevent over-fitting of machine learning models. 

Regularization is not free however. While regularization reduces the **variance** in the model results, it introduces **bias**. Whereas, an over-fit model exhibits low bias but the variance is high. The high variance leads to unpredictable results when the model is exposed to new data cases. On the other hand, the stronger the regularization of a model the lower the variance, but the greater the bias. This all means that when applying regularization you will need to contend with the **bias-variance trade-off**. 

To better understand the bias variance trade-off consider the following examples of extreme model cases:

- If the prediction for all cases is just the mean (or median), variance is minimized. The estimate for all cases is the same, so the bias of the estimates is zero. However, there is likely considerable variance in these estimates. 
- On the other hand, consider what happens when the data are fit with a kNN model with k=1. The training data will fit this model perfectly, since there is one model coefficient per training data point. The variance will be low. However, the model will have considerable bias when applied to test data. 

In either case, these extreme models will not generalize well and will exhibit large errors on any independent test data. Any practical model must come to terms with the trade-off between bias and variance to make accurate predictions. 

To better understand this trade-off you should consider the example of the mean square error, which can be decomposed into its components. The mean square error can be written as:

$$\Delta x = E \Big[ \big(Y - \hat{f}(X) \big)^2 \Big] = \frac{1}{N} \sum_{i=1}^N \big(y_i - \hat{f}(x_i) \big)^2 $$

Where,
$Y = $ the label vector.  
$X = $ the feature matrix.   
$\hat{f}(x) = $ the trained model.   

Expanding the representation of the mean square error:

$$\Delta x = \big( E[ \hat{f}(X)] - \hat{f}(X) \big)^2 + E \big[ ( \hat{f}(X) - E[ \hat{f}(X)])^2 \big] + \sigma^2\\
\Delta x = Bias^2 + Variance + Irreducable\ Error$$

Study this relationship. Notice that as regularization reduces variance, bias increases. The irreducible error will remain unchanged. Regularization parameters are chosen to minimize $\Delta x$. In many cases, this will prove challenging. 

## Load a  data set

With the above bit of theory in mind, it is time to try an example. In this example you will compute and compare linear regression models using different levels and types of regularization. 

Execute the code in the cell below to load the packages required for the rest of this notebook.

In [1]:
## Import packages
library(ggplot2)
library(repr)
library(dplyr)
library(caret)
library(glmnet)

options(repr.plot.width=4, repr.plot.height=4) # Set the initial plot area dimensions


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: lattice
Loading required package: Matrix
Loading required package: foreach
Loaded glmnet 2.0-13



The code in the cell below loads the dataset which was prepared in a previous lab. Execute this code and ensure that the expected columns are present. 

In [2]:
auto_prices = read.csv('Auto_Prices_Preped.csv')
print(dim(auto_prices))
names(auto_prices)

[1] 195  26


[1] "symboling"         "make"              "fuel.type"        
 [4] "aspiration"        "num.of.doors"      "body.style"       
 [7] "drive.wheels"      "engine.location"   "wheel.base"       
[10] "length"            "width"             "height"           
[13] "curb.weight"       "engine.type"       "num.of.cylinders" 
[16] "engine.size"       "fuel.system"       "bore"             
[19] "stroke"            "compression.ratio" "horsepower"       
[22] "peak.rpm"          "city.mpg"          "highway.mpg"      
[25] "price"             "log_price"

Notice that there are 195 cases and two label columns. 

## Split the dataset

You must now create randomly sampled training and test data sets. The `createDataPartition` function from the R caret package is used  to create indices for the training data sample. In this case 75% of the data will be used  for training the model. Since this data set is small, only 48 cases will be included in the test dataset. Execute this code and note the dimensions of the resulting data frame.

In [3]:
set.seed(1955)
## Randomly sample cases to create independent training and test data
partition = createDataPartition(auto_prices[,'fuel.type'], times = 1, p = 0.75, list = FALSE)
training = auto_prices[partition,] # Create the training sample
dim(training)
test = auto_prices[-partition,] # Create the test sample
dim(test)

[1] 147  26

[1] 48 26

## Scale numeric features

Numeric features must be rescaled so they have a similar range of values. Rescaling prevents features from having an undue influence on model training simply because then have a larger range of numeric variables. 

The code in the cell below uses the `preProcess` function from the caret function. The processing is as follows:
1. The preprocessing model object is computed. In this case the processing includes centering and scaling the numeric feature. Notice that this model is fit only to the training data.
2. The scaling is applied to both the test and training partitions.

Execute the code. 

In [4]:
num_cols = c('wheel.base', 'length', 'width', 'height', 'curb.weight', 'engine.type', 
             'num.of.cylinders', 'engine.size', 'fuel.system', 'bore', 'stroke', 
             'compression.ratio', 'horsepower', 'peak.rpm', 'city.mpg')
preProcValues <- preProcess(training[,num_cols], method = c("center", "scale"))

training[,num_cols] = predict(preProcValues, training[,num_cols])
test[,num_cols] = predict(preProcValues, test[,num_cols])
head(training[,num_cols])

,wheel.base,length,width,height,curb.weight,engine.type,num.of.cylinders,engine.size,fuel.system,bore,stroke,compression.ratio,horsepower,peak.rpm,city.mpg
1,-1.66334530,-0.4791643,-0.8655069,-2.1632960,-0.07121387,dohc,three_four,-0.01271573,mpfi,0.4347487,-1.8385856,-0.2953773,0.1417937,-0.2071160,-0.6352759
2,-1.66334530,-0.4791643,-0.8655069,-2.1632960,-0.07121387,dohc,three_four,-0.01271573,mpfi,0.4347487,-1.8385856,-0.2953773,0.1417937,-0.2071160,-0.6352759
6,0.07441198,0.1737428,0.1267119,-0.3189213,-0.14739526,ohc,five_six,0.12405588,mpfi,-0.5901228,0.4874471,-0.4195707,0.1160607,0.8284638,-0.9389687
7,1.00535338,1.3566568,2.4268556,0.7962820,0.47877864,ohc,five_six,0.12405588,mpfi,-0.5901228,0.4874471,-0.4195707,0.1160607,0.8284638,-0.9389687
8,1.00535338,1.3566568,2.4268556,0.7962820,0.68316774,ohc,five_six,0.12405588,mpfi,-0.5901228,0.4874471,-0.4195707,0.1160607,0.8284638,-0.9389687
9,1.00535338,1.3566568,2.4268556,0.8820669,0.92843467,ohc,five_six,0.01007954,mpfi,-0.8097382,0.4874471,-0.4692480,0.8880485,0.8284638,-1.2426616


## A first linear regression model

To create a baseline for comparison, you will first create a model using all 45 features and no regularization. In the terminology used before this model has high variance and low bias. In other words, this model is over-fit. 

The code in the cell below should be familiar. In summary, it performs the following processing:
1. Define and train the linear regression model using the training features and labels. The model is defined using the R modeling language.
2. Score the model using the test feature set. 
3. Compute and display key performance metrics for the model using the test feature set. 
4. Plot a histogram of the residuals of the model using the test partition.
5. Plot a Q-Q Normal plot of the residuals of the model using the test partition.
6. Plot the residuals of the model vs. the predicted values using the test partition. 

Execute this code and examine the results for the linear regression model. 

In [5]:
print_metrics = function(lin_mod, df, score, label){
    resids = df[,label] - score
    resids2 = resids**2
    N = length(score)
    r2 = as.character(round(summary(lin_mod)$r.squared, 4))
    adj_r2 = as.character(round(summary(lin_mod)$adj.r.squared, 4))
    cat(paste('Mean Square Error      = ', as.character(round(sum(resids2)/N, 4)), '\n'))
    cat(paste('Root Mean Square Error = ', as.character(round(sqrt(sum(resids2)/N), 4)), '\n'))
    cat(paste('Mean Absolute Error    = ', as.character(round(sum(abs(resids))/N, 4)), '\n'))
    cat(paste('Median Absolute Error  = ', as.character(round(median(abs(resids)), 4)), '\n'))
    cat(paste('R^2                    = ', r2, '\n'))
    cat(paste('Adjusted R^2           = ', adj_r2, '\n'))
}

hist_resids = function(df, score, label, bins = 10){
    options(repr.plot.width=4, repr.plot.height=3) # Set the initial plot area dimensions
    df$resids = df[,label] - score
    bw = (max(df$resids) - min(df$resids))/(bins + 1)
    ggplot(df, aes(resids)) + 
       geom_histogram(binwidth = bw, aes(y=..density..), alpha = 0.5) +
       geom_density(aes(y=..density..), color = 'blue') +
       xlab('Residual value') + ggtitle('Histogram of residuals')
}

resids_qq = function(df, score, label){
    options(repr.plot.width=4, repr.plot.height=3.5) # Set the initial plot area dimensions
    df$resids = df[,label] - score
    ggplot() + 
    geom_qq(data = df, aes(sample = resids)) + 
    ylab('Quantiles of residuals') + xlab('Quantiles of standard Normal') +
    ggtitle('QQ plot of residual values')
}


resid_plot = function(df, score, label){
    df$score = score
    df$resids = df[,label] - score
    ggplot(df, aes(score, resids)) + 
    geom_point() + 
    ggtitle('Residuals vs. Predicted Values') +
    xlab('Predicted values') + ylab('Residuals')
}

lin_mod = lm(log_price ~ make + fuel.type + aspiration + num.of.doors + body.style +
             drive.wheels + engine.location + wheel.base + length +
             width + height + curb.weight + num.of.cylinders +
             engine.size + bore + stroke + compression.ratio +
             horsepower + peak.rpm + city.mpg, data = training)

score = predict(lin_mod, newdata = test)
print_metrics(lin_mod, test, score, label = 'log_price')      
hist_resids(test, score, label = 'log_price')   
resids_qq(test, score, label = 'log_price')
resid_plot(test, score, label = 'log_price')

Mean Square Error      =  0.0269 
Root Mean Square Error =  0.164 
Mean Absolute Error    =  0.1299 
Median Absolute Error  =  0.1035 
R^2                    =  0.9715 
Adjusted R^2           =  0.9592 


Overall these results are reasonably good. The error metrics are relatively small. Further, the distribution of the residuals is a bit skewed, but otherwise well behaved. There is one notable outlier.

## Create model matrix

To explore the bias-variance trade-off for l1 and l2 regularization, you work with the R `glmnet` model. The `glmnet` model will not work with data frames. Rather, this model function requires a numeric matrix for the training features and a vector of labels. 

To create model matrix the code in the cell below uses the `dummyVars` function from the caret package. A predict method is applied to create numeric model matrices for training and test. Execute the code and examine the resulting matrix.

In [6]:
cols = c('make', 'fuel.type', 'aspiration', 'num.of.doors', 'body.style',
         'drive.wheels', 'engine.location', 'wheel.base', 'length',
         'width', 'height', 'curb.weight', 'num.of.cylinders', 'engine.size', 
         'bore', 'stroke', 'compression.ratio', 'horsepower', 
         'peak.rpm', 'city.mpg', 'log_price')

dummies <- dummyVars(log_price ~ ., data = auto_prices[,cols])
training_dummies = predict(dummies, newdata = training[,cols])
print(dim(training_dummies))
head(training_dummies)

[1] 147  52


,make.alfa-romero,make.audi,make.bmw,make.chevrolet,make.dodge,make.honda,make.isuzu,make.jaguar,make.mazda,make.mercedes-benz,...,num.of.cylinders.eight_twelve,num.of.cylinders.five_six,num.of.cylinders.three_four,engine.size,bore,stroke,compression.ratio,horsepower,peak.rpm,city.mpg
1,1,0,0,0,0,0,0,0,0,0,...,0,0,1,-0.01271573,0.4347487,-1.8385856,-0.2953773,0.1417937,-0.2071160,-0.6352759
2,1,0,0,0,0,0,0,0,0,0,...,0,0,1,-0.01271573,0.4347487,-1.8385856,-0.2953773,0.1417937,-0.2071160,-0.6352759
6,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0.12405588,-0.5901228,0.4874471,-0.4195707,0.1160607,0.8284638,-0.9389687
7,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0.12405588,-0.5901228,0.4874471,-0.4195707,0.1160607,0.8284638,-0.9389687
8,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0.12405588,-0.5901228,0.4874471,-0.4195707,0.1160607,0.8284638,-0.9389687
9,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0.01007954,-0.8097382,0.4874471,-0.4692480,0.8880485,0.8284638,-1.2426616


Notice that there is now one **dummy variable** for each category of the categorical variables. Only one dummy variable is coded with a one for each set of categories. This is known as **one hot encoding**. By using numeric dummy variable, the entire training feature array is now numeric. 

Execute the code in the cell below to encode the test features as a dummy variable matrix.

In [7]:
test_dummies = predict(dummies, newdata = test[,cols])

## Apply l2 regularization

Now, you will apply **l2 regularization** to constrain the model parameters. Constraining the model parameters prevent over-fitting of the model. This method is also known as **Ridge Regression**. 

But, how does this work? l2 regularization applies a **penalty** proportional to the **l2** or **Euclidean norm** of the model weights to the loss function. For linear regression using squared error as the metric, the total **loss function** is the sum of the squared error and the regularization term. The total loss function can then be written as follows:  

$$J(\beta) = ||A \beta + b||^2 + \lambda ||\beta||^2$$

Where the penalty term on the model coefficients, $\beta_i$, is written:

$$\lambda || \beta||^2 = \lambda \big(\beta_1^2 + \beta_2^2 + \ldots + \beta_n^2 \big)^{\frac{1}{2}} = \lambda \Big( \sum_{i=1}^n \beta_i^2 \Big)^{\frac{1}{2}}$$

We call $||\beta||^2$ the **l2 norm** of the coefficients, since we raise the weights of each coefficient to the power of 2, sum the squares, and then raise the sum to the power of $\frac{1}{2}$. 

You can think of this penalty as constraining the 12 or Euclidean norm of the model weight vector. The value of $\lambda$ determines how much the norm of the coefficient vector constrains the solution. You can see a geometric interpretation of the l2 penalty constraint in the figure below.  

<img src="img/L2.jpg" alt="Drawing" style="width:750px; height:400px"/>
<center> **Geometric view of l2 regularization**

Notice that for a constant value of the l2 norm, the values of the model parameters $B_1$ and $B_2$ are related. The Euclidean or l2 norm of the coefficients is shown as the dotted circle. The constant value of the l2 norm is a constant value of the penalty. Along this circle the coefficients change in relation to each other to maintain a constant l2 norm. For example, if $B_1$ is maximized then $B_2 \sim 0$, or vice versa. It is important to note that l2 regularization is a **soft constraint**. Coefficients are driven close to, but likely not exactly to, zero.    

With this bit of theory in mind, it is time to try an example of l2 regularization. The code in the cell below performs the following  processing:
1. Constructs a glmnet model using the feature matrix and vector of labels with the following arguments:
  - The `nlambda` argument determines the number of regularization parameters to be tested. 
  - `alpha` determines the weighting between l1 (alpha = 1) and l2 regularization (alpha = 0). 
  - A `gaussian` distribution family is used for the response since this is a regression problem and the residuals are expected to have a Gaussian or Normal distribution. 
2. A plot is created of the model parameters vs. the regularization constraint hyperparameter, lambda. 

Execute this code and examine the results. 

In [8]:
options(repr.plot.width=6, repr.plot.height=6) # Set the initial plot area dimensions

glmnet_mod_l2 = glmnet(x = training_dummies, y = training[,'log_price'], 
                       nlambda = 20, alpha = 0, family = 'gaussian')
plot(glmnet_mod_l2, xlab = 'Inverse of regulariation')

Next, execute the code in the cell below to compute a cross validation of the model with different regularization hyperparameters and display the result. Cross validation is discussed in another lab. 

In [9]:
cv_fit = cv.glmnet(x = training_dummies, y = training[,'log_price'], 
                       nlambda = 20, alpha = 0, family = 'gaussian')
plot(cv_fit)

Examine these results.

The first plot shows the value of each model coefficient vs. the regularization hyperparameter. The hyperparamter increases to the left. On the right the hyperparameter is small and the model is under-constrained. The model parameters have a wide range of values. On the left the regularization hyperparameter is at a maximum and the model coefficients are constrained to near zero. Notice how each coefficient value smoothly decrease toward zero. There are a few coefficients which increase in value for some part of their trajectories. This behavior results from the fact that l2 regularization is a soft constraint on the coefficient values. 

The second plot shows the RMSE from the cross validation vs. the log of the hyperparameter. Notice that in this chart the maximum of the hyperparameter is on the right, following the usual convention. As the regularization is increased to the right, model variance decreases but with higher bias. Less regularization produces a model with less bias but greater variance. The dotted vertical lines indicate that the optimal hyperparameter is at the low end of the range. 

Next, you will evaluate the model using the best l2 regularization hyperparameter discovered above. The code in the cell below computes predicted values from the optimal l2 regression model using the test data. The predict method for `glmnet` returns predictions for each value of the hyperparameter.  In this case the 18th value is chosen based on the analysis above. Performance metrics and diagnostic plots are then displayed. 

Execute the code and answer **Question 1** on the course page.

In [10]:
print_metrics_glm = function(df, score, label){
    resids = df[,label] - score
    resids2 = resids**2
    N = length(score)
    SSR = sum(resids2)
    SST = sum((mean(df[,label]) - df[,label])**2)
    r2 = as.character(round(1 - SSR/SST, 4))
    cat(paste('Mean Square Error      = ', as.character(round(sum(resids2)/N, 4)), '\n'))
    cat(paste('Root Mean Square Error = ', as.character(round(sqrt(sum(resids2)/N), 4)), '\n'))
    cat(paste('Mean Absolute Error    = ', as.character(round(sum(abs(resids))/N, 4)), '\n'))
    cat(paste('Median Absolute Error  = ', as.character(round(median(abs(resids)), 4)), '\n'))
    cat(paste('R^2                    = ', r2, '\n'))
}

score = predict(glmnet_mod_l2, newx = test_dummies)[,18]

print_metrics_glm(test, score, 'log_price')
hist_resids(test, score, label = 'log_price')   
resids_qq(test, score, label = 'log_price')
resid_plot(test, score, label = 'log_price')

Mean Square Error      =  0.0182 
Root Mean Square Error =  0.1347 
Mean Absolute Error    =  0.0986 
Median Absolute Error  =  0.0634 
R^2                    =  0.9176 


Compare the error metrics achieved to those of the un-regularized model. The error metrics for the regularized model are somewhat better. This fact, indicates that the regularized model generalizes better than the un-regularized model. Notice also that the residuals are a bit closer to Normally distributed than for the un-regularized model. 

## Apply l1 regularizaton

Regularization can be performed using norms other than l2. The **l1 regularization** or **Lasso**  method limits the sum of the absolute values of the model coefficients. The l1 norm is sometime know as the **Manhattan norm**, since distance are measured as if you were traveling on a rectangular grid of streets. This is in contrast to the l2 norm that measures distance 'as the crow flies'. 

We can compute the l1 norm of the model coefficients as follows:

$$||\beta||^1 = \big( |\beta_1| + |\beta_2| + \ldots + |\beta_n| \big) = \Big( \sum_{i=1}^n |\beta_i| \Big)^1$$

where $|\beta_i|$ is the absolute value of $\beta_i$. 

Notice that to compute the l1 norm, we raise the sum of the absolute values to the first power.

As with l2 regularization, for l1 regularization, a penalty term is multiplied by the l1 norm of the model coefficients. A penalty multiplier, $\lambda$, determines how much the norm of the coefficient vector constrains values of the weights. The complete loss function is the sum of the squared errors plus the penalty term which becomes: 

$$J(\beta) = ||A \beta + b||^2 + \lambda ||\beta||^1$$

You can see a geometric interpretation of the l1 norm penalty in the figure below.  

<img src="img/L1.jpg" alt="Drawing" style="width:700px; height:400px"/>
<center> **Geometric view of L1 regularization**

The l1 norm is constrained by the sum of the absolute values of the coefficients. This fact means that values of one parameter highly constrain another parameter. The dotted line in the figure above looks as though someone has pulled a rope or lasso around pegs on the axes. This behavior leads the name lasso for l1 regularization.  

Notice that in the figure above that if $B_1 = 0$ then $B_2$ has a value at the limit, or vice versa. In other words, using a l1 norm constraint forces some weight values to zero to allow other coefficients to take non-zero values. Thus, you can think of the l1 norm constraint **knocking out** some weights from the model altogether. In contrast to l2 regularization, l1 regularization does drive some coefficients to exactly zero.

The code in the cell below computes l1 regularized or lasso regression over a grid of regularization values. The `alpha` hyperparameter of the `glmnet` model is a pure l1 regularization model. Execute the code and examine the results.

In [11]:
options(repr.plot.width=6, repr.plot.height=6) # Set the initial plot area dimensions

glmnet_mod_l1 = glmnet(x = training_dummies, y = training[,'log_price'], 
                       nlambda = 20, alpha = 1, family = 'gaussian')
plot(glmnet_mod_l1, xlab = 'Inverse of regulariation')

cv_fit = cv.glmnet(x = training_dummies, y = training[,'log_price'], 
                       nlambda = 20, alpha = 1, family = 'gaussian')
plot(cv_fit)

The two plots created are the same types as used for the l2 regularization example. 

The first plot shows the value of each model coefficient vs. the regularization hyperparameter. The hyperparamter increases to the left. On the right the hyperparameter is small and the model is under-constrained. The model parameters have a wide range of values. On the left the regularization hyperparameter is at a maximum and the model coefficients are constrained to near zero. Notice how the coefficient values are abruptly driven to zero as the hyperparameter increases. There are a few coefficients which increase in value for some part of their trajectories. These increases are abrupt as well, driven by another coefficient becoming zero. This behavior results from the fact that l1 regularization is a hard constraint on the coefficient values. 

The second plot shows the RMSE from the cross validation vs. the log of the hyperparameter. Notice that in this chart the maximum of the hyperparameter is on the right, following the usual convention. As the regularization is increased to the right, model variance decreases but with higher bias. Less regularization produces a model with less bias but greater variance. The dotted vertical lines indicate that the optimal hyperparameter is at the middle of the range. 

Next, you will evaluate the model using the best l1 regularization parameter discovered above. The code in the cell below computes predicted values from the optimal l1 regression model using the test data. The predict method for `glmnet` returns predictions for each value of the hyperparameter.  In this case the 13th value is chosen based on the analysis above. Performance metrics and diagnostic plots are then displayed. 

Execute the code and answer **Question 2** on the course page.

In [12]:
score = predict(glmnet_mod_l1, newx = test_dummies)[,13]
print_metrics_glm(test, score, 'log_price')
hist_resids(test, score, label = 'log_price')   
resids_qq(test, score, label = 'log_price')
resid_plot(test, score, label = 'log_price')

Mean Square Error      =  0.0188 
Root Mean Square Error =  0.137 
Mean Absolute Error    =  0.1095 
Median Absolute Error  =  0.0813 
R^2                    =  0.9148 


Compare the error metrics achieved to those of the un-regularized model. The error metrics for the regularized model are somewhat better. This fact, indicates that the regularized model generalizes better than the un-regularized model. Notice also that the residuals are a bit closer to Normally distributed than for the un-regularized model.

## Summary

In this lab you have explored the basics of regularization. Regularization can prevent machine learning models from being over-fit. Regularization is required to help machine learning models generalize when placed in production. Selection of regularization strength involves consideration of the bias-variance trade-off. As the regularization is increased, model variance decreases but with higher bias. Less regularization produces a model with less bias but greater variance.  

L2 and l1 regularization constrain model coefficients to prevent over-fitting. L2 regularization constrains model coefficients using a Euclidean norm. L2 regularization can drive some coefficients toward zero, usually not to zero. On the other hand, l1 regularization can drive model coefficients to zero. 

An optimal model can use a weighted mix of l1 and l2 regularization. 